In [ ]:
import os
import json
import random
import cv2
import numpy as np

# Step 1: Extract images and annotations from the zip file

# Step 2: Load VGG formatted JSON annotations
with open('C:\\Users\\Shoaib\\Downloads\\cat_dog (1)\\cat_dog\\cat_dog_annotations.json', 'r') as f:
    vgg_annotations = json.load(f)

# Step 3: Define resize scale percentage range
resize_scale_range = (60, 100)

# Step 4: Define brightness augmentation range
brightness_range = (-30, 30)

# Step 5: Define rotation angles
rotation_angles = range(0, 360, 5)

# Step 6: Create COCO formatted annotations dictionary
coco_annotations = {
    'info': {},
    'licenses': [],
    'images': [],
    'annotations': [],
    'categories': []
}

# Step 7: Iterate over each image and its annotations
for image_data in vgg_annotations['C:\\Users\\Shoaib\\Downloads\\cat_dog (1)\\cat_dog']:
    image_id = image_data['id']
    image_path = image_data['C:\\Users\\Shoaib\\Downloads\\cat_dog (1)']
    annotations = [ann for ann in vgg_annotations['annotations'] if ann['image_id'] == image_id]

    # Step 8: Load the image
    image = cv2.imread(image_path)

    # Step 9: Resize and augment the image for each rotation angle
    for angle in rotation_angles:
        # Step 9a: Resize the image
        resize_scale = random.uniform(*resize_scale_range)
        resized_image = cv2.resize(image, None, fx=resize_scale, fy=resize_scale)

        # Step 9b: Adjust bounding box coordinates for the resized image
        for annotation in annotations:
            # Adjust bounding box coordinates
            x, y, w, h = annotation['bbox']
            x *= resize_scale
            y *= resize_scale
            w *= resize_scale
            h *= resize_scale
            annotation['bbox'] = [x, y, w, h]

        # Step 9c: Add random brightness augmentation
        brightness = random.uniform(*brightness_range)
        augmented_image = resized_image + brightness

        # Step 9d: Rotate the image
        rotation_matrix = cv2.getRotationMatrix2D((augmented_image.shape[1] / 2, augmented_image.shape[0] / 2), angle, 1)
        rotated_image = cv2.warpAffine(augmented_image, rotation_matrix, (augmented_image.shape[1], augmented_image.shape[0]))

        # Step 10: Save the resulting image to disk
        output_path = f'results/image_{image_id}_angle_{angle}.jpg'
        cv2.imwrite(output_path, rotated_image)

        # Step 11: Update COCO formatted annotations
        coco_annotations['images'].append({
            'id': len(coco_annotations['images']) + 1,
            'file_name': output_path,
            'height': rotated_image.shape[0],
            'width': rotated_image.shape[1]
        })

        for annotation in annotations:
            coco_annotations['annotations'].append(annotation)

# Step 12: Save the COCO annotations as a JSON file
with open('coco_annotations.json', 'w') as f:
    json.dump(coco_annotations, f)

# Step 13: Compress the augmented images and the COCO annotations JSON file into a zip file
# (You'll need to use a suitable library to create the zip file)
